In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
# seed 값 설정

seed=0
np.random.seed(seed)
tf.random.set_seed(3)

In [3]:
# mnist 데이터셋 불러오기
(x_train,y_class_train),(x_test,y_class_test)=tf.keras.datasets.mnist.load_data()

print('학습셋 이미지수: {}' .format(x_train.shape[0])) # minist 손글씨 학습셋 이미지셋 은 (60000,28,28) 로 구성되어 있다
print('테스트셋 이미지수: {}' .format(x_test.shape[0]) )

학습셋 이미지수: 60000
테스트셋 이미지수: 10000


In [4]:
# 차원 변환 과정

x_train=x_train.reshape(x_train.shape[0],28,28,1).astype(float) / 255 # 이제 주어진 가로 28, 세로 28개의 2차원 배열을 784개의 1차원 배열로 바꿔 줘야 한다. # 흰색이 0이고 글쓰기 1~255까지이기 때문에 0부터 1 사이를 만들기 위해서 255으로 나눠 준다. 컨볼루션층이나 맥스풀링은 주어진 이미지를 2차원 배열로 다룬다는 점이다. 활성화 함수는 1차원 배열로 다룬다. float32와 float32를 비교해 보았는데 별 차이 없는거 같다. mnist_simpe.ipynb는 x_train.shape[0],784).astype(float)로 했는데 이 파일에서는 x_train.shape[0],28,28,1로 했다. 아마도 conv2d와 maxfooling2d 때문인거 같다. 다른 문서를 찾아 보니 (18724, 28, 28) (18724,)이기 때문에 변경해 주어야 한다고 되어 있다 (my_image.py 참고)
print(x_train.shape)

x_test=x_test.reshape(x_test.shape[0],28,28,1).astype(float) / 255
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [5]:
y_class_train=pd.get_dummies(y_class_train)
y_class_test=pd.get_dummies(y_class_test)


print(y_class_train.shape)
print(y_class_test.shape)

(60000, 10)
(10000, 10)


In [6]:
x=tf.keras.layers.Input([28,28,1])
h=tf.keras.layers.Conv2D(6, kernel_size=5, padding='same', activation='swish')(x)
h=tf.keras.layers.MaxPool2D()(h) # 위치의 크기를 절반으로 줄인다. 여기서는 28*28이 14*14로 변한다
h=tf.keras.layers.Conv2D(16,kernel_size=5,activation='swish')(h) # 이미지가 feature map(특징 맵)이 16장이 생긴다
h=tf.keras.layers.MaxPool2D()(h)
h=tf.keras.layers.Dropout(0.25)(h) # 노드가 많아지거나 층이  많아진다고 학습이 무조건 좋아지는 것이 아닌 과적합이  될수 있다. drop out은 은닉층에 배치된 노드 중 일부를 임의로 꺼 주는 것이다. dropout(0.25)를 하면 25% 노드를 일부러 꺼 주는 것이다

h=tf.keras.layers.Flatten()(h) # 컨볼루션층이나 맥스풀링은 주어진 이미지를 2차원 배열로 다룬다는 점이다. 활성화 함수는 1차원 배열로 다룬다.
h=tf.keras.layers.Dense(120, activation='swish')(h)
h=tf.keras.layers.Dense(84, activation='swish')(h)

h=tf.keras.layers.Dropout(0.25)(h) # 노드가 많아지거나 층이  많아진다고 학습이 무조건 좋아지는 것이 아닌 과적합이  될수 있다. drop out은 은닉층에 배치된 노드 중 일부를 임의로 꺼 주는 것이다. dropout(0.25)를 하면 25% 노드를 일부러 꺼 주는 것이다

y=tf.keras.layers.Dense(10,activation='softmax')(h)

model=tf.keras.models.Model(x,y)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')



In [7]:

from keras.callbacks import EarlyStopping
Early_Stopping_callbacks=EarlyStopping(monitor='accuracy', patience=10) # 모니터할 값과 테스트 오차가 좋아지지 않아도 몇번까지 기다릴지 결정한다


In [8]:
model.fit(x_train,y_class_train, epochs=100, batch_size=200, callbacks=[Early_Stopping_callbacks])

model.summary()




Epoch 1/100
300/300 [==============================] - 8s 23ms/step - loss: 0.5004 - accuracy: 0.8474
Epoch 2/100
300/300 [==============================] - 7s 23ms/step - loss: 0.1383 - accuracy: 0.9582
Epoch 3/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0980 - accuracy: 0.9697
Epoch 4/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0755 - accuracy: 0.9766
Epoch 5/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0603 - accuracy: 0.9812
Epoch 6/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0547 - accuracy: 0.9837
Epoch 7/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0473 - accuracy: 0.9850
Epoch 8/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0417 - accuracy: 0.9872
Epoch 9/100
300/300 [==============================] - 7s 23ms/step - loss: 0.0377 - accuracy: 0.9883
Epoch 10/100
300/300 [==============================] - 7s 24ms/step - loss: 0.035